In [1]:
import SimpleITK as sitk
import matplotlib.pyplot as plt

from ipywidgets import widgets
from ipywidgets import interact


def plot_scans(img_arr_list, title_list):
    cols = len(img_arr_list)
    num_slices = img_arr_list[0].shape[0]

    def callback(z=None):
        fig, ax = plt.subplots(1, cols, figsize=(3*cols, 3*cols), squeeze=False)
        for idx in range(cols):
            ax[0][idx].imshow(img_arr_list[idx][z, :, :], cmap='gray')
            ax[0][idx].title.set_text(title_list[idx])
            ax[0][idx].axis('off')
            ax[0][idx].grid(False)

        fig.tight_layout()
        fig.show()

    interact(
        callback,
        z=widgets.IntSlider(value=0, min=0, max=(num_slices - 1), step=1)
    )

In [3]:
import torch

In [4]:
import pyrootutils
import hydra

root = pyrootutils.setup_root(search_from="./", indicator=[".git", "pyproject.toml"], pythonpath=True, dotenv=True)


In [5]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
hydra.initialize("../configs/datamodule/", version_base="1.2")
data_module = hydra.compose("fpr_mae_datamodule.yaml")

In [6]:
datamodule = hydra.utils.instantiate(data_module)

Unable to find logging config yaml. Configuring fallback values
[2023-11-16 11:56:45 +0530] [WARNING] Invalid or unsupported selector 'select:not(:has(option[selected])) option:first-of-type,
option[selected]:not(option[selected] ~ option[selected]) ', ('Unknown pseudo-class', 'has')


2023-11-16 11:56:58.021 | INFO     | src.datamodules.base_dataloader:setup:56 - Train set Length: 10344
2023-11-16 11:56:58.022 | INFO     | src.datamodules.base_dataloader:setup:57 - Val set Length: 1799
2023-11-16 11:56:58.023 | INFO     | src.datamodules.base_dataloader:setup:64 - Test set Length: 227


In [8]:
train_ds = datamodule.train_ds
val_ds = datamodule.val_ds
test_ds = datamodule.test_ds

In [9]:
len(train_ds)

10344

In [117]:
i = 22334
# data = test_ds[i]
data = train_ds[i]
image = data['image'].squeeze(0)

In [9]:
image = data['image'].squeeze(0)

In [10]:
image.shape

torch.Size([16, 48, 48])

In [11]:
import cv2
import numpy as np

# Assuming 'tensor' is your 16x48x48 tensor
smoothed_tensor = np.zeros_like(image)

for i in range(image.shape[0]):  # Iterate through each slice
    smoothed_tensor[i, :, :] = cv2.GaussianBlur(image[i, :, :].numpy(), (3, 3), 0)


In [12]:
smoothed_tensor.shape

(16, 48, 48)

In [48]:
image.shape

torch.Size([16, 48, 48])

In [13]:
plot_scans([image , smoothed_tensor] , ["" ,""])

interactive(children=(IntSlider(value=0, description='z', max=15), Output()), _dom_classes=('widget-interact',…

In [14]:
from src.common.nn_modules.nets.mae.mae3d import MAE3D

In [10]:
from src.common.nn_modules.nets.segmentation.multitaskfusion import modify_model_state_dict

In [11]:
ckpt_path = "/data_nas5/qer/shubham/mae3d_checkpoints/fpr_mae/runs/2023-11-15_10-15-31/checkpoints/epoch=94_step=49875_val_loss=0.66.ckpt"


In [24]:
# hydra.core.global_hydra.GlobalHydra.instance().clear()
# hydra.initialize("../configs/model/", version_base="1.2")
# model_config = hydra.compose("mae3d.yaml")

/home/users/shubham.kumar/miniconda3/envs/qct_deep/lib/python3.9/site-packages/hydra/_internal/defaults_list.py:251: UserWarning:

In 'mae3d.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information



In [12]:
import yaml

with open("/data_nas5/qer/shubham/mae3d_checkpoints/fpr_mae/runs/2023-11-15_10-15-31/tensorboard/version_0/hparams.yaml", 'r') as file:
    data = yaml.safe_load(file)



In [13]:
model_config = data['model']['model']

In [14]:
model_config

{'_target_': 'src.common.nn_modules.nets.mae.mae3d.MAE3D',
 'pos_embed_type': 'sincos',
 'mask_ratio': 0.7,
 'input_size': [48, 48, 16],
 'patch_size': [6, 6, 4],
 'in_chans': 1,
 'encoder_embed_dim': 576,
 'encoder_depth': 6,
 'encoder_num_heads': 12,
 'decoder_embed_dim': 384,
 'decoder_depth': 4,
 'decoder_num_heads': 12}

In [15]:
model = hydra.utils.instantiate(model_config)

/home/users/shubham.kumar/miniconda3/envs/qct_deep/lib/python3.9/site-packages/torch/functional.py:568: UserWarning:

torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/aten/src/ATen/native/TensorShape.cpp:2228.)



In [16]:
model

MAE3D(
  (encoder): MAEViTEncoder(
    (patch_embed): PatchEmbed3D(
      (proj): Conv3d(1, 576, kernel_size=(6, 6, 4), stride=(6, 6, 4))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((576,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=576, out_features=1728, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=576, out_features=576, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((576,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=576, out_features=2304, bias=True)
          (act): GELU()
          (drop1): Dropout(p=0.0, inplace=False)
          (fc2): Linear(in_features=2304, out_features=576, bias=True)
          (drop2): D

In [17]:
k= torch.load(ckpt_path, map_location = "cpu")
state_dict = k['state_dict']

In [18]:
k.keys()

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'MixedPrecisionPlugin'])

In [19]:
new_state_dict = modify_model_state_dict(state_dict , "model." , "")


In [20]:
model.load_state_dict(new_state_dict)

<All keys matched successfully>

In [42]:
i = 339
# data = test_ds[i]
data = val_ds[i]
image = data['image'].squeeze(0)

In [43]:
data['label'].numpy()[0]

1

In [44]:
x = image.unsqueeze(0).unsqueeze(0)

In [45]:
x = x.permute(0, 1, 3, 4, 2)

In [46]:
x.shape

torch.Size([1, 1, 48, 48, 16])

In [47]:
out = model.forward(x, return_image= True)

In [48]:
H, W, D = [48,48,16]  # Original input dimensions
Ph, Pw, Pd = [6,6,4]  # Patch dimensions

# Compute the number of patches along each dimension
num_patches_h = H // Ph
num_patches_w = W // Pw
num_patches_d = D // Pd

In [49]:
unshuffled_all_x = out[1].reshape(1, num_patches_h, num_patches_w, num_patches_d, 1, Ph, Pw, Pd)
unshuffled_all_x = unshuffled_all_x.permute(0, 4, 1, 5, 2, 6, 3, 7).reshape(1, 1, H, W, D)
final_x = unshuffled_all_x.squeeze()
final_x = final_x.permute(2, 0, 1)

In [50]:
unshuffled_all_recon = out[2].reshape(1, num_patches_h, num_patches_w, num_patches_d, 1, Ph, Pw, Pd)
unshuffled_all_recon = unshuffled_all_recon.permute(0, 4, 1, 5, 2, 6, 3, 7).reshape(1, 1, H, W, D)
final_recon = unshuffled_all_recon.squeeze()
final_recon = final_recon.permute(2, 0, 1)

In [51]:
final_recon.shape

torch.Size([16, 48, 48])

In [52]:
plot_scans([final_x , final_recon],["" , ""])

interactive(children=(IntSlider(value=0, description='z', max=15), Output()), _dom_classes=('widget-interact',…

In [63]:
out[3].shape

torch.Size([1, 256, 144])

In [ ]:
plot_scans([])

In [34]:
import torch
from safetensors import safe_open


In [35]:
path = "/cache/fast_data_nas8/qct/shubham/fpr_cache_lidc/1.3.6.1.4.1.14519.5.2.1.6279.6001.991510467496831189331312523884.safetensors"

with safe_open(path, framework="pt") as f:
    tensor_slice = f.get_slice("labels")



In [54]:

import pandas as pd

In [76]:
df = pd.read_csv("/home/users/shubham.kumar/projects/qct_training_framework/notebooks/fpr_nlst.csv")

In [77]:
df.loc[df["Status"] == "test" , "Status"] = "train"

In [78]:
df.to_csv("/home/users/shubham.kumar/projects/qct_training_framework/notebooks/fpr_nlst.csv" , index =False)

In [74]:
df

,SeriesUID,Status
0,1.2.840.113654.2.55.63958747859170795163875361...,train
1,1.2.840.113654.2.55.15001367599869928485750524...,train
2,1.2.840.113654.2.55.10331884898838628773326988...,train
3,1.2.840.113654.2.55.15545085579499669960904803...,train
4,1.2.840.113654.2.55.38607123543534460221157070...,train
...,...,...
344,1.3.6.1.4.1.14519.5.2.1.7009.9004.362228162468...,train
345,1.2.840.113654.2.55.12192207809952073415380123...,train
346,1.2.840.113654.2.55.25275808408796599515059773...,train
347,1.2.840.113654.2.55.30582414251353258917076621...,train


In [97]:
pd.read_csv("/home/users/shubham.kumar/projects/qct_training_framework/notebooks/fpr_lidc_raw.csv")

,SeriesUID,Status
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.955688628308...,train
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.216882370221...,train
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.168985655485...,train
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.313835996725...,train
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.255999614855...,train
...,...,...
878,1.3.6.1.4.1.14519.5.2.1.6279.6001.292994770358...,train
879,1.3.6.1.4.1.14519.5.2.1.6279.6001.112767175295...,train
880,1.3.6.1.4.1.14519.5.2.1.6279.6001.257383535269...,train
881,1.3.6.1.4.1.14519.5.2.1.6279.6001.173556680294...,train


In [ ]:
def func(array , lung_mask_arr) :
    
    (48*48*16)
    

In [3]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
hydra.initialize("../configs/datamodule/", version_base="1.2")
data_module = hydra.compose("lung_lobe_segmentation_datamodule.yaml")

In [4]:
datamodule = hydra.utils.instantiate(data_module)

Unable to find logging config yaml. Configuring fallback values
[2023-11-15 13:34:59 +0530] [WARNING] Invalid or unsupported selector 'select:not(:has(option[selected])) option:first-of-type,
option[selected]:not(option[selected] ~ option[selected]) ', ('Unknown pseudo-class', 'has')


2023-11-15 13:34:59.757 | INFO     | src.datamodules.load_data.lung_lobe_segmentation:lung_segmentation_data_loader:227 - train /home/users/shubham.kumar/projects/lung_lobe_segmentation/lung_lobe_cct_data.csv
2023-11-15 13:35:00.789 | INFO     | src.datamodules.load_data.lung_lobe_segmentation:_load_hdf5_seg_dataset_ctloader:284 - 291
2023-11-15 13:35:00.797 | INFO     | src.datamodules.load_data.lung_lobe_segmentation:lung_segmentation_data_loader:227 - train /home/users/shubham.kumar/projects/lung_lobe_segmentation/LCTSC_dataset.csv
2023-11-15 13:35:01.358 | INFO     | src.datamodules.load_data.lung_lobe_segmentation:_load_hdf5_seg_dataset_ctloader:284 - 24
2023-11-15 13:35:01.366 | INFO     | src.datamodules.load_data.lung_lobe_segmentation:lung_segmentation_data_loader:227 - train /home/users/shubham.kumar/projects/lung_lobe_segmentation/LOLA_dataset.csv
2023-11-15 13:35:01.708 | INFO     | src.datamodules.load_data.lung_lobe_segmentation:_load_hdf5_seg_dataset_ctloader:284 - 31
20

In [16]:
train_ds = datamodule.train_ds
val_ds = datamodule.val_ds
test_ds = datamodule.test_ds

In [17]:
import torch

In [6]:
i = 0 
data = train_ds[i]

In [7]:
image = data["image"]
lung_mask = data["lung_label"]

In [15]:
plot_scans([d, lung_mask] , ["", ""])

interactive(children=(IntSlider(value=0, description='z', max=120), Output()), _dom_classes=('widget-interact'…

In [21]:
from safetensors.torch import save_file
from tqdm.auto import tqdm
save_root = "/cache/fast_data_nas8/qct/shubham/fpr_lung_lobe_dataset"

for i in tqdm(range(len(test_ds))) : 
    
    data = test_ds[i]
    sid = data["series_uid"]

    data_to_save = {}
    data_to_save["image"] = data["image"].to(torch.int64)
    data_to_save["lung_mask"] = data["lung_label"].to(torch.int8)

    save_file(data_to_save, f"{save_root}/{sid}.safetensors")

  0%|          | 0/231 [00:00<?, ?it/s]

In [53]:
import numpy as np

def generate_3d_patch_indices(image_shape, patch_size, overlap):
    H, W, D = image_shape
    h, w, d = patch_size
    o_h, o_w, o_d = overlap

    # Calculate stride based on overlap
    s_h = h - o_h
    s_w = w - o_w
    s_d = d - o_d

    # Ensure that stride is not less than 1
    s_h, s_w, s_d = max(1, s_h), max(1, s_w), max(1, s_d)

    # Generating ranges for start indices
    z_starts = np.arange(0, H - h + 1, s_h)
    y_starts = np.arange(0, W - w + 1, s_w)
    x_starts = np.arange(0, D - d + 1, s_d)

    # Creating meshgrid of start indices
    z_grid, y_grid, x_grid = np.meshgrid(z_starts, y_starts, x_starts, indexing='ij')

    # Flattening the arrays to create a list of start indices
    starts = np.vstack([z_grid.ravel(), y_grid.ravel(), x_grid.ravel()]).T

    # Calculating the corresponding end indices
    ends = starts + np.array([h, w, d])

    # Combining start and end indices
    patch_indices = np.hstack([starts, ends])

    return patch_indices

# Example usage:
# indices = generate_3d_patch_indices((H, W, D), (h, w, d), (o_h, o_w, o_d))


In [56]:
a = generate_3d_patch_indices((350,350,200), (48,48,16), (4,4,2))

In [57]:
len(a)

686

In [58]:
a

array([[  0,   0,   0,  48,  48,  16],
       [  0,   0,  14,  48,  48,  30],
       [  0,   0,  28,  48,  48,  44],
       ...,
       [264, 264, 154, 312, 312, 170],
       [264, 264, 168, 312, 312, 184],
       [264, 264, 182, 312, 312, 198]])

In [ ]:
/data_nas5/qer/shubham/mae3d_checkpoints/fpr_mae/runs/2023-11-14_11-44-42/checkpoints/